In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

import refinitiv.dataplatform.eikon as ek
import option_ric_tools as ort

ek.set_app_key('DEFAULT_CODE_BOOK_APP_KEY')

ERROR!!! An error occurred while requesting URL('http://localhost:9000/api/status').
     ConnectError('All connection attempts failed')
ERROR!!! An error occurred while requesting URL('http://localhost:9060/api/status').
     ConnectError('All connection attempts failed')
Error: no proxy address identified.
Check if Desktop is running.
ERROR!!! An error occurred while requesting URL('http://localhost:9000/api/handshake').
     ConnectError('All connection attempts failed')
ERROR!!! An error occurred while requesting URL('http://localhost:9000/api/status').
     ConnectError('All connection attempts failed')
ERROR!!! An error occurred while requesting URL('http://localhost:9060/api/status').
     ConnectError('All connection attempts failed')
Error: no proxy address identified.
Check if Desktop is running.
ERROR!!! An error occurred while requesting URL('http://localhost:9000/api/handshake').
     ConnectError('All connection attempts failed')


Let's start by defining what data we will be fetching. We will need to define the following:
* The ISIN for the underlying security as a string
* The maturity date for any options as a string
* The strike prices for which to fetch options prices as a list of integers
* The kinds of options (call/put) to fetch prices for as a list of strings (allowed values are 'C' and 'P')

An example for IBM is provided below. If you are unsure what strikes to fetch you might want to check the price of the underlying 90 days prior to your chosen maturity. Any configurations for which no prices are available will be skipped.

In [ ]:
isin = 'US4592001014'
maturities = ['2025-9-12','2025-9-19','2025-9-26','2025-11-28','2025-11-21','2025-10-31','2025-10-24', '2025-11-14', '2025-10-10','2025-10-17']
strikes = [115, 120, 125, 130, 135]
kinds = ['C', 'P']

The code below will then try to fetch pricing for what we defined. Using the Eikon API requires quite a lot of preprocessing, which is taken care of in the file 'option_ric_tools.py' The price series for each option will be added as a column.

In [ ]:
# Get RIC for the underlying (same for all maturities)
ric = ek.get_symbology(symbol=isin, from_symbol_type='ISIN', to_symbol_type='RIC').iloc[0,0]

# Number of days of data to include (approximately 45 days)
target_days = 45

# Loop through each maturity
for maturity in maturities:
    print(f'\n{"="*60}')
    print(f'Processing maturity: {maturity}')
    print(f'{"="*60}')
    
    # Calculate the date range for approximately 45 days before maturity
    maturity_date = pd.Timestamp(maturity)
    today = pd.Timestamp(datetime.now().date())
    
    # End date is the earlier of maturity date or today
    end_date = min(maturity_date, today)
    # Start date is 45 days before end date
    start_date = end_date - timedelta(days=target_days)
    
    sdate = start_date.strftime('%Y-%m-%d')
    edate = end_date.strftime('%Y-%m-%d')
    
    print(f'Target date range: {sdate} to {edate} (approximately {target_days} days)')
    
    price_dfs = []
    for kind in kinds:
        assert kind in ['C', 'P']
        for strike in strikes:
            kind_str = 'call' if kind == 'C' else 'put'
            print(f'\nFetching prices for {kind_str} option with strike price {strike}')
            rics, prices_list = ort.get_optionRic(isin, maturity, strike, kind)
            try:
                price_df = prices_list[0]
                price_df = price_df.drop(['BID', 'ASK'], axis=1)
                price_df.columns = [f'{kind}{strike}']
                price_dfs.append(price_df)
            except IndexError:
                print('Skipping empty combination')
    
    if not price_dfs:
        print(f'No data found for maturity {maturity}, skipping...')
        continue
    
    price_df = pd.concat(price_dfs, axis=1)
    
    # Filter data to the target date range (approximately 45 days)
    price_df = price_df[(price_df.index >= start_date) & (price_df.index <= end_date)]
    
    if price_df.empty:
        print(f'No data available in the target date range for maturity {maturity}, skipping...')
        continue
    
    # Fetch underlying stock prices for the same date range
    stock = ek.get_timeseries(
        rics=[ric],
        fields=['CLOSE'],
        start_date=sdate,
        end_date=edate,
        interval='daily'
    )
    
    stock = stock.reset_index()
    stock.columns.name = None
    stock.index = stock['Date']
    stock.index.name = None
    stock = stock.drop('Date', axis=1)
    stock.columns = ['Underlying']
    
    # Filter stock data to match the same date range
    stock = stock[(stock.index >= start_date) & (stock.index <= end_date)]
    
    # Join underlying with options data
    price_df = stock.join(price_df)
    
    # Remove any rows with all NaN values (in case of date mismatches)
    price_df = price_df.dropna(how='all')
    
    # Check actual number of days in the dataset
    actual_days = len(price_df)
    print(f'\nActual number of trading days in dataset: {actual_days}')
    
    # Save to CSV with date in filename
    price_df.index.name = 'Date'
    price_df = price_df.reset_index()
    
    # Format date for filename (replace dashes with underscores)
    date_str = maturity.replace('-', '_')
    filename = f'data/options_data_{date_str}.csv'
    price_df.to_csv(filename, index=False)
    print(f'Saved data to {filename}')


Fetching prices for call option with strike price 115


EndpointError: Error code -1 | Session is not opened. Can't send any request

In [ ]:
# All data has been saved to CSV files in the data folder
# Each file is named: options_data_YYYY_MM_DD.csv
print("Processing complete! Check the data folder for CSV files.")

,C115,C120,C125,C130,C135,P115,P120,P125,P130,P135
2021-12-20,<NA>,9.81,6.9,4.44,2.81,2.84,3.95,6.6,9.45,<NA>
2021-12-21,<NA>,11.6,7.91,5,3.15,2.05,3.1,4.9,7.45,<NA>
2021-12-22,<NA>,11.6,8.06,5.2,3.15,1.77,2.85,4.36,6.9,<NA>
2021-12-23,<NA>,12.7,8.4,5.7,3.45,1.52,2.44,3.75,5.95,8.87
2021-12-27,<NA>,13.05,9.1,5.75,3.5,1.36,2.12,3.45,5.4,8.5


The code above now processes all maturities in the list. For each maturity:
* Fetches option prices for all strike prices and option types
* Fetches the underlying stock prices for the same time period
* Saves the combined data to a CSV file in the data folder with the format: `options_data_YYYY_MM_DD.csv`

In [ ]:
# RIC is now fetched once at the beginning of the loop (in cell 4)
# This cell is no longer needed

ERROR!!! An error occurred while requesting URL('http://localhost:9000/api/udf').
     ConnectError('All connection attempts failed')


ConnectError: All connection attempts failed

The underlying stock prices are now fetched within the loop for each maturity.

In [ ]:
# Stock data fetching is now done within the loop (in cell 4)
# This cell is no longer needed

In [ ]:
# Data preview is no longer needed here since each maturity is saved separately
# You can load any CSV file to preview the data

,Underlying,C115,C120,C125,C130,C135,P115,P120,P125,P130,P135
2021-12-20,127.06,<NA>,9.81,6.9,4.44,2.81,2.84,3.95,6.6,9.45,<NA>
2021-12-21,128.97,<NA>,11.6,7.91,5,3.15,2.05,3.1,4.9,7.45,<NA>
2021-12-22,129.75,<NA>,11.6,8.06,5.2,3.15,1.77,2.85,4.36,6.9,<NA>
2021-12-23,130.63,<NA>,12.7,8.4,5.7,3.45,1.52,2.44,3.75,5.95,8.87
2021-12-27,131.62,<NA>,13.05,9.1,5.75,3.5,1.36,2.12,3.45,5.4,8.5


Each maturity's data is automatically saved to a CSV file in the data folder. The files are named with the format: `options_data_YYYY_MM_DD.csv`

In [ ]:
# CSV saving is now done within the loop (in cell 4)
# This cell is no longer needed